# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U azure-identity azure-search-documents azure-ai-documentintelligence langchain langchain-community langchain-openai

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_AI_SEARCH_ENDPOINT = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_ADMIN_KEY = os.getenv("AZURE_AI_SEARCH_ADMIN_KEY")
AZURE_DOCUMENTINTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENTINTELLIGENCE_ENDPOINT")
AZURE_DOCUMENTINTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENTINTELLIGENCE_API_KEY")

# AI Enrichment
Azure 의 Document Intelligence 는 비정형 문서에서 텍스트, 테이블, 폼 등을 추출하여 구조화하는데 필수적인 AI 서비스이다. PDF 등과 같은 문서를 ETL pipeline 을 태울 때 유용하게 사용된다.

## Loading Document
Native 하게 Azure SDK 를 써도 되지만, langchain 이 abstraction 을 잘해놓아서 이를 활용해보자.

In [2]:
import os
from langchain_community.document_loaders.doc_intelligence import AzureAIDocumentIntelligenceLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=os.environ["AZURE_DOCUMENTINTELLIGENCE_ENDPOINT"],
    api_key=os.environ["AZURE_DOCUMENTINTELLIGENCE_API_KEY"],
    file_path="./resources/KB주택시장리뷰_2025년 10월호.pdf",
    api_model="prebuilt-layout",
)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(docs)


In [3]:
# metadata 에 있는 값들을 document 에 들어가기에 중복되는 key 들을 제거한다.
for d in chunks:
    d.metadata.pop("content", None)          # 충돌 키 제거
    d.metadata.pop("content_vector", None)   # (안전)
    d.metadata.pop("id", None)

## Upload Documents
위에서 parsing 한 chunks 들을 embeddings 해서 AI Search 로 업로드하자.

In [4]:
index_name = "housing"

emb = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
)

# 여기선 index name 이 없을 때 schema 를 추론하여 자동 생성해준다.
vs = AzureSearch(
    azure_search_endpoint=AZURE_AI_SEARCH_ENDPOINT,
    azure_search_key=AZURE_AI_SEARCH_ADMIN_KEY,
    index_name=index_name,
    embedding_function=emb.embed_query,
)
vs.add_documents(documents=chunks)

['NDA5ZDA1NDEtNTM1Ny00ODdiLTkyMjYtNTE2M2E3YTE5MmQ0',
 'MzFlNTg2ZTctOTI0Ny00NmI1LWI3MGYtODAyOGI3MmM1OTc0',
 'ZjFhYTAwYzMtN2UzMy00NzBhLWJmN2YtZjQ3MGJkMjQ4ZGJm',
 'MWIzNzQxMmItMTgxMC00YzA0LThkNWItMGNiN2M4MzZiZGEy',
 'OTEyYjdmOWYtNTk0Yy00MGFjLThlMTMtMDNkYTQ3NWM4ZDJh',
 'MWMyNDNmOWMtZDcyOC00Yzc0LTk5ODQtOGUxZWE2M2Q4NmZl',
 'MjQ4MWJiNjEtYzQwMS00OTJjLWE0YzMtMmM2Mjc4ZTU0NmYx',
 'ZmRlOGIzMDYtMTQ2Yy00MzQzLWE2MGMtMjZjODhkODE5M2Zi',
 'OGY4OTc4OTMtN2E5Yy00M2RmLWE1NmItYmQ5NzQ2M2IxNjAz',
 'MDZjYzY5MjAtNzUxYS00MWVlLWE2NTAtODM2ZGE0ZjA1Zjg3',
 'MzJlZGMyOTAtNjBlNy00ZWFmLWI1ZDAtNDNkOTE5ODczOGRh',
 'ZTNhM2MzMGYtMTY5OC00NjIxLTliMjYtYzk5ZTY5ODM1ZWRh']

## Process search query
한번 search methods 를 사용해보자.

In [ ]:
# Perform a similarity search
docs = vs.similarity_search(
    query="매매가격 상승폭이 가장 컸던 지역은?",
    k=3,
    search_type="hybrid",
)
for idx, doc in enumerate(docs):
    print(f"\n=== Document {idx + 1} ===")
    print(doc.page_content)

• 연초 이후 완만한 감소세를 보이던 미분양 아파트는 8월 6.6만 호로 전월 대비 4.4천 호 증가. 이는 비수도권
주택 경기가 여전히 좋지 않은 상황에서 하반기 분양물량이 증가한 데 기인


## ○ [주택금융] 대출 규제 영향으로 주택 거래량이 감소하면서 주택담보대출은 증가폭 축소

• 9월 은행권 주택담보대출 잔액은 932.7조 원으로 전월에 비해 증가 규모가 감소(8월 +3.8조 원→9월 +2.5조 원).
6.27대책 영향과 전세자금 수요도 감소하면서 증가폭이 축소

• 8월 예금은행 주택담보대출 금리(신규 취급액 기준)는 3.96%, 7월 주택담보대출 연체율은 0.29%를 기록

<!-- PageFooter="Kb KB 금융지주 경영연구소" -->
<!-- PageNumber="3" -->
<!-- PageBreak -->

<!-- PageHeader="KB 금융지주 경영연구소" -->


# [매매가격] 수도권 가격 상승세 지속되며 주택시장 양극화 심화


## ■ 9월 전국 주택 매매가격은 전월 대비 0.08% 상승

○ 수도권(서울 0.52%, 경기 0.11%)을 중심으로 매매가격은 상승세 지속(8월 0.18%→9월 0.24%)

• 서울 아파트 매매가격 상승세가 확대되는 가운데, 토지거래허가구역에 포함되지 않은 성동구·
마포구 · 양천구를 중심으로 매매가격 상승세 지속

※ 8월 25일→9월 22일: 성동구 0.17%→0.73%, 마포구 0.16%→0.53%, 양천구0.16%→0.41%

○ 반면 5개광역시(8월 -0.14%→9월 -0.19%)와 기타지방(-0.05%→-0.01%)은 매매가격 하락폭 확대


<figure>
<figcaption>그림 1| 지역별 주택 매매가격지수 변동률 추이</figcaption>

1.0

(전월 대비, %)

0.5

0.0

-0.5

-1.0

전국

수도권

-1.5

5개광역시

기타지방

-2.0

'22.9

123.3

23.9

'24.3

'24.9

'25.3

'25.9
<figure

## Delete the index
사용하지 않는다면, index 는 삭제하자.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(
    endpoint=AZURE_AI_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_AI_SEARCH_ADMIN_KEY),
).delete_index(index_name)